In [10]:
# 必要なライブラリのインポート
import requests
import pandas as pd
from pandas import DataFrame as df
from bs4 import BeautifulSoup

In [25]:
#とりあえずはGARB Tokyoの夜のみ口コミの部分だけやってみた
url = 'https://tabelog.com/rst/rstdtl_rvwlst/?pal=tokyo&rcd=13026436&srt=&sby=&smp=1&use_type=2&rvw_part=all&lc=2'
res = requests.get(url)
content = res.content
soup = BeautifulSoup(content, 'lxml')

#他のとこでも同じURLが取れるはずなんですが、うまくできなくてこっちになりました
#弊害として
customers = soup.find_all('a', class_='rvw-item__title-target')

reveiw_url = list()
for  customer in customers:
        c=customer.get("href")
        reveiw_url.append(c)
        
#これで口コミごとのURLがゲットできた
url_df= 'https://tabelog.com' + pd.Series(reveiw_url)

In [5]:
#ここら辺の流れは大西くんと同じかんじでやっていく
#applyとmapが混ざっているのに意味はないです、requestはapplyの方が早かった気がする。。
#mapとapply大好きマン

all_text = url_df.apply(lambda x : requests.get(x))
all_content = all_text.map(lambda x : x.content)
all_soup = all_content.map(lambda x : BeautifulSoup(x, 'lxml'))
all_review = all_soup.apply(lambda x : x.find_all('div', class_="rvw-item__rvw-comment" ))

In [17]:
#まだ口コミの文字列は取れていないのでさらに削っていく
start_num  = all_review.apply(lambda x : str(x).find('<p>') )
end_num = all_review.apply(lambda x : str(x).find('</p>') )

#<p></p>で囲まれた部分が口コミ欄だったのでそこだけ抜き取る
part_review = all_review.apply(lambda x : str(x)[start_num[len(x)]: end_num[len(x)]])

In [18]:
review_contents =[]
for i in range(len(part_review)):
    dic =str.maketrans("", "", "<br/>\n</p><p></div>[]")
    nakami = part_review[i].translate(dic)  
    #複数回replaceするのが億劫になった
    #ta =part_review[i].replace('<br/>', ' ')
    if i ==0:
        f = open('test.txt', 'w')  #新規書き込み
        f.write(nakami)
    else:
        f = open('test.txt', 'a')  #追加書き込み
        f.writelines(nakami)

In [19]:
cat test.txt

          平日のディナーにて予約をせず訪問。丸ビルを出て、お店を探していたところこちらのお店が目に入ったので、そのまま入店。テラスと店内を選ぶことが出来ます、今回は店内へ。店内は意外と広く、ソファー席やテーブル席、ハイカウンターなどがあります。早速、ビールで乾杯し、以下を注文。・シマアジのカルパッチョ 1,300 円本日のメニューより注文。綺麗に飾り付けられたカルパッチョ、美味しい。・グリーンサラダ  800円レタス、ズッキーニ、ブロッコリーなどゴロゴロ野菜も入っており食べ応えもあり美味しい。ドレッシングも女性が好みそうな味で、美味しかったです。・マグロほほ肉の白ワインラグータリアテッレ 1,300円マグロほほ肉、トマトが入っているシンプルなパスタですが、とっても美味しい‼︎味付けも抜群で、ワインにもぴったり。がっつり食事というよりは、雰囲気の良いお店なので軽く飲んだり食べたりするにはオススメです。また、仕事終わりにふらっと立ち寄りたいと思います。                  ***2016年6月***再訪です♪って前回書いたの4年前か～(*_*)びっくり。今回はピクルス、海老のアヒージョ、茄子と鰯のトマト煮的なもの、カルボナーラなどをワイン片手にいただきまして、どれも美味しく(*^^*)特にアヒージョ☆いい味だしてます。お値段はさすがに4年前に比べると価格アップしていたけれどもこの内容でこの価格なら満足、という感想は以前と違わずでした。20台前半のカップルが多かったような？年齢層が更にさがった感が。。。。って自分があがったから？笑店内の音楽はもう少し小さめにかけていても良いカモ・・・・丸の内仲通りにあるだけに、海外っぽいムードで美味しく飲食できる良店だと思います。ご馳走さまでした！！**2012年7月***午後六時頃。特急まで二時間半あり時間を潰したいけどガッツリコースを食べるにはお昼に食べ過ぎたし。。。そんな時たまたま通りがかりました。今までは通りがかる度二次会を開催されていたので、なんとなく気になりつつ来店できていなかったお店ですが、この日は通常営業。メニューを見るとタパスもいろいろ、リゾットもある(#^.^#)てことでお店の中へ。若い人達が多いかな？都会的な雰囲気。隣の席では合コンしてました。笑でも、ワイワイ騒ぐ感じではなく大人な合コン。笑昼

やったね！